In [ ]:
"h2"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import COBYLA

# Import GPU accelerated estimator
from qiskit_aer.primitives import Estimator as AerEstimator

# PySCF for classical references
from pyscf import gto, scf, fci

class H2Simulator:
    """GPU-accelerated H₂ simulator."""
    
    def __init__(self, verbose=True):
        self.verbose = verbose
        self.results = {}
        
        # Best settings for H₂
        self.basis = "cc-pVDZ"
        self.charge = 0
        self.spin = 0
        self.active_space = None  # No active space needed for H₂
        
        if self.verbose:
            print(f"\n{'='*50}")
            print("H₂ GPU-Accelerated VQE Simulation")
            print(f"{'='*50}")
            print(f"Basis: {self.basis}")
            print(f"Using GPU acceleration with qiskit-aer-gpu-cu11")
            print(f"{'='*50}\n")
    
    def get_h2_geometry(self, distance):
        """Get H₂ geometry with variable bond length."""
        return [
            ("H", (0.0, 0.0, 0.0)),
            ("H", (0.0, 0.0, distance))
        ]
    
    def build_pyscf_mol(self, geometry):
        """Create PySCF molecule."""
        atom_spec = ""
        for atom in geometry:
            atom_spec += f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}; "
        
        mol = gto.Mole()
        mol.atom = atom_spec.strip("; ")
        mol.basis = self.basis
        mol.charge = self.charge
        mol.spin = self.spin
        mol.build()
        return mol
    
    def build_qiskit_problem(self, geometry):
        """Create Qiskit problem."""
        atom_spec = ""
        for atom in geometry:
            atom_spec += f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}; "
        
        driver = PySCFDriver(
            atom=atom_spec.strip("; "),
            basis=self.basis,
            unit=DistanceUnit.ANGSTROM,
            charge=self.charge,
            spin=self.spin
        )
        return driver.run()
    
    def run_vqe_gpu(self, problem):
        """Run VQE with GPU acceleration."""
        # Create mapper
        mapper = ParityMapper(num_particles=problem.num_particles)
        
        # Create circuit components
        init_state = HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper
        )
        
        ansatz = UCCSD(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
            initial_state=init_state
        )
        
        # Random initialization
        np.random.seed(42)
        initial_point = 0.1 * np.random.random(ansatz.num_parameters)
        
        # Create GPU-accelerated estimator
        try:
            estimator = AerEstimator(
                run_options={"shots": 8192},
                approximation=True,
                device="GPU"
            )
            if self.verbose:
                print("Using GPU acceleration")
        except Exception as e:
            if self.verbose:
                print(f"GPU acceleration failed: {e}")
                print("Falling back to CPU")
            estimator = AerEstimator(run_options={"shots": 8192})
        
        # Optimizer
        optimizer = COBYLA(maxiter=200)
        
        # Create and run VQE
        vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point)
        qubit_op = mapper.map(problem.hamiltonian.second_q_op())
        
        if self.verbose:
            print(f"Ansatz parameters: {ansatz.num_parameters}")
            print(f"Qubit operator size: {qubit_op.num_qubits} qubits")
        
        # Run VQE with timing
        start_time = time.time()
        result = vqe.compute_minimum_eigenvalue(qubit_op)
        runtime = time.time() - start_time
        
        # Get energy
        energy = problem.interpret(result).total_energies[0].real
        
        if self.verbose:
            print(f"VQE completed in {runtime:.2f} seconds")
            print(f"Energy: {energy:.8f} Hartree")
        
        return energy, result, runtime
    
    def run_references(self, mol):
        """Run classical reference calculations."""
        results = {}
        
        # HF
        try:
            mf = scf.RHF(mol)
            results["hf"] = mf.kernel()
        except Exception as e:
            print(f"HF failed: {e}")
            results["hf"] = None
        
        # FCI
        try:
            mf_fci = scf.RHF(mol).run()
            cisolver = fci.FCI(mol, mf_fci.mo_coeff)
            results["fci"], _ = cisolver.kernel()
        except Exception as e:
            print(f"FCI failed: {e}")
            results["fci"] = None
        
        return results
    
    def run_energy_scan(self, start=0.4, end=3.0, points=14):
        """Run potential energy surface scan."""
        distances = np.linspace(start, end, points)
        
        vqe_energies = []
        hf_energies = []
        fci_energies = []
        runtimes = []
        
        for i, dist in enumerate(distances):
            print(f"\nPoint {i+1}/{len(distances)} at distance {dist:.2f} Å")
            
            # Get geometry and problem
            geometry = self.get_h2_geometry(dist)
            
            # Run references
            mol = self.build_pyscf_mol(geometry)
            refs = self.run_references(mol)
            hf_energies.append(refs.get("hf"))
            fci_energies.append(refs.get("fci"))
            
            # Run VQE
            try:
                problem = self.build_qiskit_problem(geometry)
                energy, _, runtime = self.run_vqe_gpu(problem)
                vqe_energies.append(energy)
                runtimes.append(runtime)
                
                print(f"  H-H: {dist:.3f} Å | VQE: {energy:.6f} | Runtime: {runtime:.2f}s")
                if refs.get("hf"):
                    print(f"  HF: {refs['hf']:.6f} | Correlation: {energy - refs['hf']:.6f}")
                if refs.get("fci"):
                    print(f"  FCI: {refs['fci']:.6f} | Error: {(energy - refs['fci'])*1000:.3f} mHa")
                
            except Exception as e:
                print(f"VQE failed: {e}")
                vqe_energies.append(np.nan)
                runtimes.append(np.nan)
        
        # Store results
        self.results = {
            "distances": distances,
            "vqe": vqe_energies,
            "hf": hf_energies,
            "fci": fci_energies,
            "runtimes": runtimes
        }
        
        return self.results
    
    def plot_results(self, save_path=None):
        """Plot energy scan results."""
        if not self.results:
            print("No results to plot")
            return
        
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), gridspec_kw={'height_ratios': [3, 1]})
        
        # Plot energies
        ax1.plot(self.results["distances"], self.results["vqe"], 'o-', label='VQE (GPU)', color='limegreen')
        if any(~np.isnan(self.results["hf"])):
            ax1.plot(self.results["distances"], self.results["hf"], 's--', label='HF', color='red')
        if any(~np.isnan(self.results["fci"])):
            ax1.plot(self.results["distances"], self.results["fci"], 'x-', label='FCI', color='black')
        
        ax1.set_xlabel("H-H Distance (Å)")
        ax1.set_ylabel("Energy (Hartree)")
        ax1.set_title(f"H₂ Potential Energy Curve ({self.basis} basis)")
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Plot runtimes
        ax2.plot(self.results["distances"], self.results["runtimes"], 'o-', color='blue')
        ax2.set_xlabel("H-H Distance (Å)")
        ax2.set_ylabel("Runtime (s)")
        ax2.set_title("VQE Runtime with GPU Acceleration")
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=300)
        else:
            plt.show()
    
    def get_summary(self):
        """Get summary of results."""
        if not self.results:
            return "No results to summarize"
        
        try:
            min_idx = np.nanargmin(self.results["vqe"])
            min_dist = self.results["distances"][min_idx]
            min_energy = self.results["vqe"][min_idx]
            avg_runtime = np.nanmean(self.results["runtimes"])
            
            summary = f"\n{'='*30} H₂ RESULTS SUMMARY {'='*30}\n"
            summary += f"Basis: {self.basis}\n"
            summary += f"Equilibrium distance: {min_dist:.3f} Å\n"
            summary += f"Minimum energy: {min_energy:.8f} Hartree\n"
            summary += f"Average GPU runtime: {avg_runtime:.2f} seconds\n"
            
            if not np.isnan(self.results["hf"][min_idx]):
                hf = self.results["hf"][min_idx]
                summary += f"HF energy: {hf:.8f} Hartree\n"
                summary += f"Correlation energy: {min_energy - hf:.8f} Hartree\n"
            
            if not np.isnan(self.results["fci"][min_idx]):
                fci = self.results["fci"][min_idx]
                summary += f"FCI energy: {fci:.8f} Hartree\n"
                summary += f"Error vs FCI: {(min_energy - fci)*1000:.3f} mHartree\n"
            
            summary += f"{'='*76}\n"
            return summary
            
        except Exception as e:
            return f"Error generating summary: {e}"


# Example usage
if __name__ == "__main__":
    print(f"Started: {datetime.now()}")
    
    # Create simulator
    h2 = H2Simulator(verbose=True)
    
    # Run energy scan
    h2.run_energy_scan(start=0.4, end=3.0, points=14)
    
    # Plot results
    h2.plot_results(save_path="h2_gpu.png")
    
    # Print summary
    print(h2.get_summary())

In [ ]:
"ne"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import BravyiKitaevMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import L_BFGS_B

# Import GPU accelerated estimator
from qiskit_aer.primitives import Estimator as AerEstimator

# PySCF for classical references
from pyscf import gto, scf, mcscf

class NeonSimulator:
    """GPU-accelerated Ne simulator."""
    
    def __init__(self, verbose=True):
        self.verbose = verbose
        self.results = {}
        
        # Best settings for Ne
        self.basis = "cc-pVDZ"
        self.charge = 0
        self.spin = 0
        self.active_space = {
            "num_electrons": (4, 4),  # 8 valence electrons
            "num_spatial_orbitals": 5,  # 5 orbitals
            "active_orbitals": [1, 2, 3, 4, 5]  # Skip 1s core orbital
        }
        
        if self.verbose:
            print(f"\n{'='*50}")
            print("Neon (Ne) GPU-Accelerated VQE Simulation")
            print(f"{'='*50}")
            print(f"Basis: {self.basis}")
            print(f"Active space: {self.active_space}")
            print(f"Using GPU acceleration with qiskit-aer-gpu-cu11")
            print(f"{'='*50}\n")
    
    def get_ne_geometry(self):
        """Get Ne atom geometry."""
        return [("Ne", (0.0, 0.0, 0.0))]
    
    def build_pyscf_mol(self):
        """Create PySCF molecule."""
        mol = gto.Mole()
        mol.atom = "Ne 0.0 0.0 0.0"
        mol.basis = self.basis
        mol.charge = self.charge
        mol.spin = self.spin
        mol.build()
        return mol
    
    def build_qiskit_problem(self):
        """Create Qiskit problem with active space."""
        driver = PySCFDriver(
            atom="Ne 0.0 0.0 0.0",
            basis=self.basis,
            unit=DistanceUnit.ANGSTROM,
            charge=self.charge,
            spin=self.spin
        )
        problem = driver.run()
        
        # Apply active space transformer
        transformer = ActiveSpaceTransformer(
            num_electrons=self.active_space["num_electrons"],
            num_spatial_orbitals=self.active_space["num_spatial_orbitals"],
            active_orbitals=self.active_space["active_orbitals"]
        )
        problem = transformer.transform(problem)
        
        if self.verbose:
            print(f"Active space: {sum(problem.num_particles)} electrons in {problem.num_spatial_orbitals} orbitals")
        
        return problem
    
    def run_vqe_gpu(self, problem):
        """Run VQE with GPU acceleration."""
        # Bravyi-Kitaev mapper is better for Ne
        mapper = BravyiKitaevMapper()
        
        # Create circuit components
        init_state = HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper
        )
        
        ansatz = UCCSD(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
            initial_state=init_state
        )
        
        # Random initialization - important for Ne
        np.random.seed(42)
        initial_point = 0.1 * np.random.random(ansatz.num_parameters)
        
        # Create GPU-accelerated estimator with more shots for Ne
        try:
            estimator = AerEstimator(
                run_options={"shots": 16384},  # More shots for Ne
                approximation=True,
                device="GPU"
            )
            if self.verbose:
                print("Using GPU acceleration")
        except Exception as e:
            if self.verbose:
                print(f"GPU acceleration failed: {e}")
                print("Falling back to CPU")
            estimator = AerEstimator(run_options={"shots": 16384})
        
        # Optimizer - L-BFGS-B works well for Ne
        optimizer = L_BFGS_B(maxiter=500)
        
        # Create and run VQE
        vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point)
        qubit_op = mapper.map(problem.hamiltonian.second_q_op())
        
        if self.verbose:
            print(f"Ansatz parameters: {ansatz.num_parameters}")
            print(f"Qubit operator size: {qubit_op.num_qubits} qubits")
        
        # Run VQE with timing
        start_time = time.time()
        result = vqe.compute_minimum_eigenvalue(qubit_op)
        runtime = time.time() - start_time
        
        # Get energy
        energy = problem.interpret(result).total_energies[0].real
        
        if self.verbose:
            print(f"VQE completed in {runtime:.2f} seconds")
            print(f"Energy: {energy:.8f} Hartree")
        
        return energy, result, runtime
    
    def run_references(self, mol):
        """Run classical reference calculations."""
        results = {}
        
        # HF
        try:
            mf = scf.RHF(mol)
            results["hf"] = mf.kernel()
        except Exception as e:
            print(f"HF failed: {e}")
            results["hf"] = None
        
        # CASSCF with same active space
        try:
            mf_cas = scf.RHF(mol).run()
            mycas = mcscf.CASSCF(
                mf_cas,
                self.active_space["num_spatial_orbitals"],
                sum(self.active_space["num_electrons"])
            )
            results["casscf"] = mycas.kernel()[0]
        except Exception as e:
            print(f"CASSCF failed: {e}")
            results["casscf"] = None
        
        return results
    
    def run_calculation(self):
        """Run single-point calculation for Ne atom."""
        print("\nRunning Ne atom calculation...")
        
        # Build molecule
        mol = self.build_pyscf_mol()
        
        # Run references
        refs = self.run_references(mol)
        
        # Run VQE
        try:
            problem = self.build_qiskit_problem()
            energy, _, runtime = self.run_vqe_gpu(problem)
            
            # Store results
            self.results = {
                "vqe": energy,
                "hf": refs.get("hf"),
                "casscf": refs.get("casscf"),
                "runtime": runtime
            }
            
            # Print results
            print("\nNeon atom energies:")
            print(f"  VQE (GPU): {energy:.8f} Hartree (runtime: {runtime:.2f}s)")
            if refs.get("hf"):
                print(f"  HF: {refs['hf']:.8f} Hartree")
                print(f"  Correlation: {energy - refs['hf']:.8f} Hartree")
            if refs.get("casscf"):
                print(f"  CASSCF: {refs['casscf']:.8f} Hartree")
                print(f"  Error vs CASSCF: {(energy - refs['casscf'])*1000:.3f} mHartree")
            
            return self.results
            
        except Exception as e:
            print(f"VQE calculation failed: {e}")
            return None
    
    def get_summary(self):
        """Get summary of results."""
        if not self.results:
            return "No results to summarize"
        
        try:
            summary = f"\n{'='*30} NEON RESULTS SUMMARY {'='*30}\n"
            summary += f"Basis: {self.basis}\n"
            summary += f"Active space: {sum(self.active_space['num_electrons'])}e in {self.active_space['num_spatial_orbitals']}o\n"
            summary += f"VQE energy: {self.results['vqe']:.8f} Hartree\n"
            summary += f"GPU runtime: {self.results['runtime']:.2f} seconds\n"
            
            if self.results.get("hf"):
                hf = self.results["hf"]
                summary += f"HF energy: {hf:.8f} Hartree\n"
                summary += f"Correlation energy: {self.results['vqe'] - hf:.8f} Hartree\n"
            
            if self.results.get("casscf"):
                casscf = self.results["casscf"]
                summary += f"CASSCF energy: {casscf:.8f} Hartree\n"
                summary += f"Error vs CASSCF: {(self.results['vqe'] - casscf)*1000:.3f} mHartree\n"
            
            summary += f"{'='*76}\n"
            return summary
            
        except Exception as e:
            return f"Error generating summary: {e}"


# Example usage
if __name__ == "__main__":
    print(f"Started: {datetime.now()}")
    
    # Create simulator
    ne = NeonSimulator(verbose=True)
    
    # Run calculation
    ne.run_calculation()
    
    # Print summary
    print(ne.get_summary())

In [ ]:
"h2o"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP

# Import GPU accelerated estimator
from qiskit_aer.primitives import Estimator as AerEstimator

# PySCF for classical references
from pyscf import gto, scf, mcscf

class WaterSimulator:
    """GPU-accelerated H₂O simulator."""
    
    def __init__(self, verbose=True):
        self.verbose = verbose
        self.results = {}
        
        # Best settings for H₂O
        self.basis = "cc-pVDZ"
        self.charge = 0
        self.spin = 0
        self.active_space = {
            "num_electrons": (4, 4),  # 8 valence electrons
            "num_spatial_orbitals": 7,  # 7 orbitals
            "active_orbitals": [1, 2, 3, 4, 5, 6, 7]  # Skip O 1s core
        }
        
        if self.verbose:
            print(f"\n{'='*50}")
            print("Water (H₂O) GPU-Accelerated VQE Simulation")
            print(f"{'='*50}")
            print(f"Basis: {self.basis}")
            print(f"Active space: {self.active_space}")
            print(f"Using GPU acceleration with qiskit-aer-gpu-cu11")
            print(f"{'='*50}\n")
    
    def get_h2o_geometry(self, bond_length):
        """
        Get H₂O geometry with variable O-H bond length.
        Angle fixed at ~104.5 degrees.
        """
        angle = 104.5 * np.pi / 180  # radians
        x = bond_length * np.sin(angle / 2)
        y = bond_length * np.cos(angle / 2)

        return [
            ("O", (0.0, 0.0, 0.0)),
            ("H", (x, y, 0.0)),
            ("H", (-x, y, 0.0))
        ]
    
    def format_geometry(self, geometry):
        """Convert geometry to string format."""
        atom_str = ""
        for atom in geometry:
            atom_str += f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}; "
        return atom_str.strip("; ")
    
    def build_pyscf_mol(self, geometry):
        """Create PySCF molecule."""
        atom_str = self.format_geometry(geometry)
        mol = gto.Mole()
        mol.atom = atom_str
        mol.basis = self.basis
        mol.charge = self.charge
        mol.spin = self.spin
        mol.build()
        return mol
    
    def build_qiskit_problem(self, geometry):
        """Create Qiskit problem with active space."""
        atom_str = self.format_geometry(geometry)
        driver = PySCFDriver(
            atom=atom_str,
            basis=self.basis,
            unit=DistanceUnit.ANGSTROM,
            charge=self.charge,
            spin=self.spin
        )
        problem = driver.run()
        
        # Apply active space transformer
        transformer = ActiveSpaceTransformer(
            num_electrons=self.active_space["num_electrons"],
            num_spatial_orbitals=self.active_space["num_spatial_orbitals"],
            active_orbitals=self.active_space["active_orbitals"]
        )
        problem = transformer.transform(problem)
        
        if self.verbose:
            print(f"Active space: {sum(problem.num_particles)} electrons in {problem.num_spatial_orbitals} orbitals")
        
        return problem
    
    def run_vqe_gpu(self, problem):
        """Run VQE with GPU acceleration."""
        # Parity mapper with symmetry reduction
        mapper = ParityMapper(num_particles=problem.num_particles)
        
        # Create circuit components
        init_state = HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper
        )
        
        ansatz = UCCSD(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
            initial_state=init_state
        )
        
        # Random initialization
        np.random.seed(42)
        initial_point = 0.1 * np.random.random(ansatz.num_parameters)
        
        # Create GPU-accelerated estimator
        try:
            estimator = AerEstimator(
                run_options={"shots": 8192},
                approximation=True,
                device="GPU"
            )
            if self.verbose:
                print("Using GPU acceleration")
        except Exception as e:
            if self.verbose:
                print(f"GPU acceleration failed: {e}")
                print("Falling back to CPU")
            estimator = AerEstimator(run_options={"shots": 8192})
        
        # SLSQP optimizer works well for water
        optimizer = SLSQP(maxiter=400)
        
        # Create and run VQE
        vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point)
        qubit_op = mapper.map(problem.hamiltonian.second_q_op())
        
        if self.verbose:
            print(f"Ansatz parameters: {ansatz.num_parameters}")
            print(f"Qubit operator size: {qubit_op.num_qubits} qubits")
        
        # Run VQE with timing
        start_time = time.time()
        result = vqe.compute_minimum_eigenvalue(qubit_op)
        runtime = time.time() - start_time
        
        # Get energy
        energy = problem.interpret(result).total_energies[0].real
        
        if self.verbose:
            print(f"VQE completed in {runtime:.2f} seconds")
            print(f"Energy: {energy:.8f} Hartree")
        
        return energy, result, runtime
    
    def run_references(self, mol):
        """Run classical reference calculations."""
        results = {}
        
        # HF
        try:
            mf = scf.RHF(mol)
            results["hf"] = mf.kernel()
        except Exception as e:
            print(f"HF failed: {e}")
            results["hf"] = None
        
        # CASSCF with same active space
        try:
            mf_cas = scf.RHF(mol).run()
            mycas = mcscf.CASSCF(
                mf_cas,
                self.active_space["num_spatial_orbitals"],
                sum(self.active_space["num_electrons"])
            )
            results["casscf"] = mycas.kernel()[0]
        except Exception as e:
            print(f"CASSCF failed: {e}")
            results["casscf"] = None
        
        return results
    
    def run_energy_scan(self, start=0.8, end=1.8, points=8):
        """Run potential energy surface scan."""
        distances = np.linspace(start, end, points)
        
        vqe_energies = []
        hf_energies = []
        casscf_energies = []
        runtimes = []
        
        for i, dist in enumerate(distances):
            print(f"\nPoint {i+1}/{len(distances)} at O-H distance {dist:.2f} Å")
            
            # Get geometry and problem
            geometry = self.get_h2o_geometry(dist)
            
            # Run references
            mol = self.build_pyscf_mol(geometry)
            refs = self.run_references(mol)
            hf_energies.append(refs.get("hf"))
            casscf_energies.append(refs.get("casscf"))
            
            # Run VQE
            try:
                problem = self.build_qiskit_problem(geometry)
                energy, _, runtime = self.run_vqe_gpu(problem)
                vqe_energies.append(energy)
                runtimes.append(runtime)
                
                print(f"  O-H: {dist:.3f} Å | VQE: {energy:.6f} | Runtime: {runtime:.2f}s")
                if refs.get("hf"):
                    print(f"  HF: {refs['hf']:.6f} | Correlation: {energy - refs['hf']:.6f}")
                if refs.get("casscf"):
                    print(f"  CASSCF: {refs['casscf']:.6f} | Error: {(energy - refs['casscf'])*1000:.3f} mHa")
                
            except Exception as e:
                print(f"VQE failed: {e}")
                vqe_energies.append(np.nan)
                runtimes.append(np.nan)
        
        # Store results
        self.results = {
            "distances": distances,
            "vqe": vqe_energies,
            "hf": hf_energies,
            "casscf": casscf_energies,
            "runtimes": runtimes
        }
        
        return self.results
    
    def plot_results(self, save_path=None):
        """Plot energy scan results."""
        if not self.results:
            print("No results to plot")
            return
        
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), gridspec_kw={'height_ratios': [3, 1]})
        
        # Plot energies
        ax1.plot(self.results["distances"], self.results["vqe"], 'o-', label='VQE (GPU)', color='limegreen')
        if any(~np.isnan(self.results["hf"])):
            ax1.plot(self.results["distances"], self.results["hf"], 's--', label='HF', color='red')
        if any(~np.isnan(self.results["casscf"])):
            ax1.plot(self.results["distances"], self.results["casscf"], 'd-', label='CASSCF', color='blue')
        
        ax1.set_xlabel("O-H Distance (Å)")
        ax1.set_ylabel("Energy (Hartree)")
        ax1.set_title(f"H₂O Potential Energy Curve ({self.basis} basis)")
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Plot runtimes
        ax2.plot(self.results["distances"], self.results["runtimes"], 'o-', color='blue')
        ax2.set_xlabel("O-H Distance (Å)")
        ax2.set_ylabel("Runtime (s)")
        ax2.set_title("VQE Runtime with GPU Acceleration")
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=300)
        else:
            plt.show()
    
    def get_summary(self):
        """Get summary of results."""
        if not self.results:
            return "No results to summarize"
        
        try:
            min_idx = np.nanargmin(self.results["vqe"])
            min_dist = self.results["distances"][min_idx]
            min_energy = self.results["vqe"][min_idx]
            avg_runtime = np.nanmean(self.results["runtimes"])
            
            summary = f"\n{'='*30} H₂O RESULTS SUMMARY {'='*30}\n"
            summary += f"Basis: {self.basis}\n"
            summary += f"Active space: {sum(self.active_space['num_electrons'])}e in {self.active_space['num_spatial_orbitals']}o\n"
            summary += f"Equilibrium O-H distance: {min_dist:.3f} Å\n"
            summary += f"Minimum energy: {min_energy:.8f} Hartree\n"
            summary += f"Average GPU runtime: {avg_runtime:.2f} seconds\n"
            
            if not np.isnan(self.results["hf"][min_idx]):
                hf = self.results["hf"][min_idx]
                summary += f"HF energy: {hf:.8f} Hartree\n"
                summary += f"Correlation energy: {min_energy - hf:.8f} Hartree\n"
            
            if not np.isnan(self.results["casscf"][min_idx]):
                casscf = self.results["casscf"][min_idx]
                summary += f"CASSCF energy: {casscf:.8f} Hartree\n"
                summary += f"Error vs CASSCF: {(min_energy - casscf)*1000:.3f} mHartree\n"
            
            summary += f"{'='*76}\n"
            return summary
            
        except Exception as e:
            return f"Error generating summary: {e}"


# Example usage
if __name__ == "__main__":
    print(f"Started: {datetime.now()}")
    
    # Create simulator
    h2o = WaterSimulator(verbose=True)
    
    # Run energy scan
    h2o.run_energy_scan(start=0.8, end=1.8, points=8)
    
    # Plot results
    h2o.plot_results(save_path="h2o_gpu.png")
    
    # Print summary
    print(h2o.get_summary())

In [ ]:
"ch4"

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP

# Import GPU accelerated estimator
from qiskit_aer.primitives import Estimator as AerEstimator

# PySCF for classical references
from pyscf import gto, scf, mcscf

class WaterSimulator:
    """GPU-accelerated H₂O simulator."""
    
    def __init__(self, verbose=True):
        self.verbose = verbose
        self.results = {}
        
        # Best settings for H₂O
        self.basis = "cc-pVDZ"
        self.charge = 0
        self.spin = 0
        self.active_space = {
            "num_electrons": (4, 4),  # 8 valence electrons
            "num_spatial_orbitals": 7,  # 7 orbitals
            "active_orbitals": [1, 2, 3, 4, 5, 6, 7]  # Skip O 1s core
        }
        
        if self.verbose:
            print(f"\n{'='*50}")
            print("Water (H₂O) GPU-Accelerated VQE Simulation")
            print(f"{'='*50}")
            print(f"Basis: {self.basis}")
            print(f"Active space: {self.active_space}")
            print(f"Using GPU acceleration with qiskit-aer-gpu-cu11")
            print(f"{'='*50}\n")
    
    def get_h2o_geometry(self, bond_length):
        """
        Get H₂O geometry with variable O-H bond length.
        Angle fixed at ~104.5 degrees.
        """
        angle = 104.5 * np.pi / 180  # radians
        x = bond_length * np.sin(angle / 2)
        y = bond_length * np.cos(angle / 2)

        return [
            ("O", (0.0, 0.0, 0.0)),
            ("H", (x, y, 0.0)),
            ("H", (-x, y, 0.0))
        ]
    
    def format_geometry(self, geometry):
        """Convert geometry to string format."""
        atom_str = ""
        for atom in geometry:
            atom_str += f"{atom[0]} {atom[1][0]} {atom[1][1]} {atom[1][2]}; "
        return atom_str.strip("; ")
    
    def build_pyscf_mol(self, geometry):
        """Create PySCF molecule."""
        atom_str = self.format_geometry(geometry)
        mol = gto.Mole()
        mol.atom = atom_str
        mol.basis = self.basis
        mol.charge = self.charge
        mol.spin = self.spin
        mol.build()
        return mol
    
    def build_qiskit_problem(self, geometry):
        """Create Qiskit problem with active space."""
        atom_str = self.format_geometry(geometry)
        driver = PySCFDriver(
            atom=atom_str,
            basis=self.basis,
            unit=DistanceUnit.ANGSTROM,
            charge=self.charge,
            spin=self.spin
        )
        problem = driver.run()
        
        # Apply active space transformer
        transformer = ActiveSpaceTransformer(
            num_electrons=self.active_space["num_electrons"],
            num_spatial_orbitals=self.active_space["num_spatial_orbitals"],
            active_orbitals=self.active_space["active_orbitals"]
        )
        problem = transformer.transform(problem)
        
        if self.verbose:
            print(f"Active space: {sum(problem.num_particles)} electrons in {problem.num_spatial_orbitals} orbitals")
        
        return problem
    
    def run_vqe_gpu(self, problem):
        """Run VQE with GPU acceleration."""
        # Parity mapper with symmetry reduction
        mapper = ParityMapper(num_particles=problem.num_particles)
        
        # Create circuit components
        init_state = HartreeFock(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper
        )
        
        ansatz = UCCSD(
            problem.num_spatial_orbitals,
            problem.num_particles,
            mapper,
            initial_state=init_state
        )
        
        # Random initialization
        np.random.seed(42)
        initial_point = 0.1 * np.random.random(ansatz.num_parameters)
        
        # Create GPU-accelerated estimator
        try:
            estimator = AerEstimator(
                run_options={"shots": 8192},
                approximation=True,
                device="GPU"
            )
            if self.verbose:
                print("Using GPU acceleration")
        except Exception as e:
            if self.verbose:
                print(f"GPU acceleration failed: {e}")
                print("Falling back to CPU")
            estimator = AerEstimator(run_options={"shots": 8192})
        
        # SLSQP optimizer works well for water
        optimizer = SLSQP(maxiter=400)
        
        # Create and run VQE
        vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point)
        qubit_op = mapper.map(problem.hamiltonian.second_q_op())
        
        if self.verbose:
            print(f"Ansatz parameters: {ansatz.num_parameters}")
            print(f"Qubit operator size: {qubit_op.num_qubits} qubits")
        
        # Run VQE with timing
        start_time = time.time()
        result = vqe.compute_minimum_eigenvalue(qubit_op)
        runtime = time.time() - start_time
        
        # Get energy
        energy = problem.interpret(result).total_energies[0].real
        
        if self.verbose:
            print(f"VQE completed in {runtime:.2f} seconds")
            print(f"Energy: {energy:.8f} Hartree")
        
        return energy, result, runtime
    
    def run_references(self, mol):
        """Run classical reference calculations."""
        results = {}
        
        # HF
        try:
            mf = scf.RHF(mol)
            results["hf"] = mf.kernel()
        except Exception as e:
            print(f"HF failed: {e}")
            results["hf"] = None
        
        # CASSCF with same active space
        try:
            mf_cas = scf.RHF(mol).run()
            mycas = mcscf.CASSCF(
                mf_cas,
                self.active_space["num_spatial_orbitals"],
                sum(self.active_space["num_electrons"])
            )
            results["casscf"] = mycas.kernel()[0]
        except Exception as e:
            print(f"CASSCF failed: {e}")
            results["casscf"] = None
        
        return results
    
    def run_energy_scan(self, start=0.8, end=1.8, points=8):
        """Run potential energy surface scan."""
        distances = np.linspace(start, end, points)
        
        vqe_energies = []
        hf_energies = []
        casscf_energies = []
        runtimes = []
        
        for i, dist in enumerate(distances):
            print(f"\nPoint {i+1}/{len(distances)} at O-H distance {dist:.2f} Å")
            
            # Get geometry and problem
            geometry = self.get_h2o_geometry(dist)
            
            # Run references
            mol = self.build_pyscf_mol(geometry)
            refs = self.run_references(mol)
            hf_energies.append(refs.get("hf"))
            casscf_energies.append(refs.get("casscf"))
            
            # Run VQE
            try:
                problem = self.build_qiskit_problem(geometry)
                energy, _, runtime = self.run_vqe_gpu(problem)
                vqe_energies.append(energy)
                runtimes.append(runtime)
                
                print(f"  O-H: {dist:.3f} Å | VQE: {energy:.6f} | Runtime: {runtime:.2f}s")
                if refs.get("hf"):
                    print(f"  HF: {refs['hf']:.6f} | Correlation: {energy - refs['hf']:.6f}")
                if refs.get("casscf"):
                    print(f"  CASSCF: {refs['casscf']:.6f} | Error: {(energy - refs['casscf'])*1000:.3f} mHa")
                
            except Exception as e:
                print(f"VQE failed: {e}")
                vqe_energies.append(np.nan)
                runtimes.append(np.nan)
        
        # Store results
        self.results = {
            "distances": distances,
            "vqe": vqe_energies,
            "hf": hf_energies,
            "casscf": casscf_energies,
            "runtimes": runtimes
        }
        
        return self.results
    
    def plot_results(self, save_path=None):
        """Plot energy scan results."""
        if not self.results:
            print("No results to plot")
            return
        
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 8), gridspec_kw={'height_ratios': [3, 1]})
        
        # Plot energies
        ax1.plot(self.results["distances"], self.results["vqe"], 'o-', label='VQE (GPU)', color='limegreen')
        if any(~np.isnan(self.results["hf"])):
            ax1.plot(self.results["distances"], self.results["hf"], 's--', label='HF', color='red')
        if any(~np.isnan(self.results["casscf"])):
            ax1.plot(self.results["distances"], self.results["casscf"], 'd-', label='CASSCF', color='blue')
        
        ax1.set_xlabel("O-H Distance (Å)")
        ax1.set_ylabel("Energy (Hartree)")
        ax1.set_title(f"H₂O Potential Energy Curve ({self.basis} basis)")
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # Plot runtimes
        ax2.plot(self.results["distances"], self.results["runtimes"], 'o-', color='blue')
        ax2.set_xlabel("O-H Distance (Å)")
        ax2.set_ylabel("Runtime (s)")
        ax2.set_title("VQE Runtime with GPU Acceleration")
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        
        if save_path:
            plt.savefig(save_path, dpi=300)
        else:
            plt.show()
    
    def get_summary(self):
        """Get summary of results."""
        if not self.results:
            return "No results to summarize"
        
        try:
            min_idx = np.nanargmin(self.results["vqe"])
            min_dist = self.results["distances"][min_idx]
            min_energy = self.results["vqe"][min_idx]
            avg_runtime = np.nanmean(self.results["runtimes"])
            
            summary = f"\n{'='*30} H₂O RESULTS SUMMARY {'='*30}\n"
            summary += f"Basis: {self.basis}\n"
            summary += f"Active space: {sum(self.active_space['num_electrons'])}e in {self.active_space['num_spatial_orbitals']}o\n"
            summary += f"Equilibrium O-H distance: {min_dist:.3f} Å\n"
            summary += f"Minimum energy: {min_energy:.8f} Hartree\n"
            summary += f"Average GPU runtime: {avg_runtime:.2f} seconds\n"
            
            if not np.isnan(self.results["hf"][min_idx]):
                hf = self.results["hf"][min_idx]
                summary += f"HF energy: {hf:.8f} Hartree\n"
                summary += f"Correlation energy: {min_energy - hf:.8f} Hartree\n"
            
            if not np.isnan(self.results["casscf"][min_idx]):
                casscf = self.results["casscf"][min_idx]
                summary += f"CASSCF energy: {casscf:.8f} Hartree\n"
                summary += f"Error vs CASSCF: {(min_energy - casscf)*1000:.3f} mHartree\n"
            
            summary += f"{'='*76}\n"
            return summary
            
        except Exception as e:
            return f"Error generating summary: {e}"


# Example usage
if __name__ == "__main__":
    print(f"Started: {datetime.now()}")
    
    # Create simulator
    h2o = WaterSimulator(verbose=True)
    
    # Run energy scan
    h2o.run_energy_scan(start=0.8, end=1.8, points=8)
    
    # Plot results
    h2o.plot_results(save_path="h2o_gpu.png")
    
    # Print summary
    print(h2o.get_summary())

In [ ]:
"n2"

In [ ]:
"""
GPU-Accelerated VQE for N₂ (Nitrogen) Molecule
Optimized for qiskit-aer-gpu-cu11 0.17.0
"""
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime
import os

from qiskit.primitives import BackendEstimator
from qiskit_aer import AerSimulator
from qiskit_algorithms import VQE, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import L_BFGS_B

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

from pyscf import gto, scf, mcscf, mp

# Set number of OpenMP threads for PySCF
import os
os.environ["OMP_NUM_THREADS"] = "4"  # Adjust based on your CPU

# Configuration for N₂ molecule
BASIS = "6-31g"  # Better than sto3g for triple bond
ACTIVE_ELECTRONS = (5, 5)  # 10 valence electrons
ACTIVE_ORBITALS = 8  # Include 2s and 2p orbitals
CORE_ORBITALS = [0, 1]  # Skip 1s core orbitals
MAX_ITER = 500  # N₂ triple bond requires more iterations

# Bond distances to scan (N-N bond in Angstrom)
DISTANCES = np.concatenate([
    np.linspace(0.9, 1.3, 7),  # More points around equilibrium (1.1 Å)
    np.linspace(1.4, 2.4, 5)   # Fewer points in dissociation region
])

def configure_gpu_backend():
    """Configure and test GPU backend."""
    try:
        backend = AerSimulator(
            method="statevector",
            device="GPU",
            precision="single",  # Use single precision for speed
            max_parallel_threads=0,  # Let CUDA decide
            max_parallel_experiments=1,  # For statevector, usually 1 is best
            cuStateVec_enable=True  # Enable NVIDIA cuStateVec library if available
        )
        
        # Test GPU backend
        result = backend.run_circuits(backend.initialize(2, 0))
        
        # Success - GPU is working
        gpu_info = backend.available_devices()[0]
        print(f"GPU acceleration active: {gpu_info}")
        return backend, True
        
    except Exception as e:
        print(f"GPU acceleration failed: {e}")
        print("Falling back to CPU backend...")
        backend = AerSimulator(
            method="statevector",
            max_parallel_threads=int(os.environ.get("OMP_NUM_THREADS", 4))
        )
        return backend, False

def get_n2_geometry(distance):
    """Generate N₂ molecule geometry at given bond distance."""
    return f"N 0 0 {-distance/2}; N 0 0 {distance/2}"

def build_qiskit_problem(distance):
    """Build Qiskit Nature problem with proper active space."""
    atom_str = get_n2_geometry(distance)
    
    # Create driver and run
    driver = PySCFDriver(
        atom=atom_str, 
        basis=BASIS, 
        unit=DistanceUnit.ANGSTROM, 
        charge=0, 
        spin=0
    )
    problem = driver.run()
    
    # Print full system information
    print(f"Full system: {problem.num_spatial_orbitals} orbitals, "
          f"{problem.num_particles} electrons")
    
    # Calculate active orbitals list
    max_orbital = CORE_ORBITALS[-1] + ACTIVE_ORBITALS + 1
    active_orbitals_list = list(range(CORE_ORBITALS[-1] + 1, max_orbital))
    
    # Apply active space transformer with explicit orbital selection
    transformer = ActiveSpaceTransformer(
        num_electrons=ACTIVE_ELECTRONS,
        num_spatial_orbitals=ACTIVE_ORBITALS,
        active_orbitals=active_orbitals_list
    )
    problem = transformer.transform(problem)
    
    print(f"Active space: {sum(problem.num_particles)} electrons in "
          f"{problem.num_spatial_orbitals} orbitals")
    
    return problem

def run_vqe(problem, backend, use_gpu=True):
    """Run VQE with optimal settings for N₂."""
    # Use parity mapper with symmetry reduction
    mapper = ParityMapper(num_particles=problem.num_particles)
    
    # Create initial state and ansatz
    init_state = HartreeFock(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper
    )
    
    ansatz = UCCSD(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
        initial_state=init_state
    )
    
    print(f"UCCSD parameters: {ansatz.num_parameters}")
    
    # Better initialization - small random values with seed for reproducibility
    np.random.seed(42)
    initial_point = 0.1 * np.random.random(ansatz.num_parameters)
    
    # L-BFGS-B works well for N₂'s triple bond
    optimizer = L_BFGS_B(maxiter=MAX_ITER)
    
    # Map to qubit operator
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    
    # Create GPU-accelerated estimator if enabled
    if use_gpu:
        estimator = BackendEstimator(backend)
    else:
        from qiskit.primitives import Estimator
        estimator = Estimator()  # Default CPU estimator
    
    # Create and run VQE
    vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point)
    
    print(f"Qubit operator size: {qubit_op.num_qubits} qubits")
    print(f"Starting VQE optimization...")
    
    # Run VQE and time it
    start_time = time.time()
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    runtime = time.time() - start_time
    
    # Get interpreted energy
    energy = problem.interpret(result).total_energies[0].real
    
    print(f"VQE completed in {runtime:.1f} seconds")
    print(f"VQE energy: {energy:.8f} Hartree")
    print(f"Optimizer evaluations: {result.optimizer_evals}")
    
    return energy, runtime, result

def run_reference_calcs(distance):
    """Run reference classical calculations."""
    atom_str = get_n2_geometry(distance)
    
    # Create molecule
    mol = gto.Mole()
    mol.atom = atom_str
    mol.basis = BASIS
    mol.charge = 0
    mol.spin = 0
    mol.build()
    
    results = {}
    
    # Run RHF
    try:
        mf = scf.RHF(mol)
        results["rhf"] = mf.kernel()
        mf_obj = mf
    except Exception as e:
        print(f"RHF calculation failed: {e}")
        results["rhf"] = np.nan
        mf_obj = None
    
    # Run MP2 for dynamic correlation
    if mf_obj is not None:
        try:
            mp2 = mp.MP2(mf_obj)
            mp2.kernel()
            # MP2 correlation energy + RHF energy
            results["mp2"] = mp2.e_corr + results["rhf"]
        except Exception as e:
            print(f"MP2 calculation failed: {e}")
            results["mp2"] = np.nan
    
    # Run CASSCF with same active space
    try:
        mf_cas = scf.RHF(mol).run()
        mycas = mcscf.CASSCF(mf_cas, 
                            ncas=ACTIVE_ORBITALS, 
                            nelecas=sum(ACTIVE_ELECTRONS))
        results["casscf"] = mycas.kernel()[0]
    except Exception as e:
        print(f"CASSCF calculation failed: {e}")
        results["casscf"] = np.nan
    
    return results

def run_n2_simulation():
    """Run full N₂ simulation with VQE and classical references."""
    print(f"\n{'='*60}")
    print(f"GPU-Accelerated VQE for N₂ (Nitrogen Molecule)")
    print(f"{'='*60}")
    print(f"Basis set: {BASIS}")
    print(f"Active space: {sum(ACTIVE_ELECTRONS)}e, {ACTIVE_ORBITALS}o")
    print(f"{'='*60}\n")
    
    # Configure GPU backend
    backend, is_gpu = configure_gpu_backend()
    
    vqe_energies = []
    vqe_times = []
    rhf_energies = []
    mp2_energies = []
    casscf_energies = []
    
    for i, dist in enumerate(DISTANCES):
        print(f"\n{'-'*30}")
        print(f"N-N Distance: {dist:.3f} Å ({i+1}/{len(DISTANCES)})")
        print(f"{'-'*30}")
        
        # Run reference calculations
        ref_results = run_reference_calcs(dist)
        rhf_energies.append(ref_results.get("rhf", np.nan))
        mp2_energies.append(ref_results.get("mp2", np.nan))
        casscf_energies.append(ref_results.get("casscf", np.nan))
        
        # Run VQE
        try:
            problem = build_qiskit_problem(dist)
            vqe_energy, vqe_time, _ = run_vqe(problem, backend, use_gpu=is_gpu)
            vqe_energies.append(vqe_energy)
            vqe_times.append(vqe_time)
            
            # Print comparison
            print("\nEnergy comparison at N-N distance = {:.3f} Å:".format(dist))
            print(f"  VQE:    {vqe_energy:.8f} Hartree (in {vqe_time:.1f}s)")
            print(f"  RHF:    {ref_results.get('rhf', 'N/A')}")
            if ref_results.get("mp2") is not None:
                print(f"  MP2:    {ref_results.get('mp2'):.8f} Hartree")
            if ref_results.get("casscf") is not None:
                print(f"  CASSCF: {ref_results.get('casscf'):.8f} Hartree")
            
            # Calculate correlation energy
            if not np.isnan(ref_results.get("rhf", np.nan)):
                corr_energy = vqe_energy - ref_results.get("rhf")
                print(f"  Correlation energy: {corr_energy:.8f} Hartree")
                
        except Exception as e:
            print(f"VQE calculation failed: {e}")
            vqe_energies.append(np.nan)
            vqe_times.append(np.nan)
    
    # Store results
    results = {
        'distances': DISTANCES,
        'vqe': vqe_energies,
        'vqe_times': vqe_times,
        'rhf': rhf_energies,
        'mp2': mp2_energies,
        'casscf': casscf_energies,
    }
    
    return results

def plot_results(results, save_path=None):
    """Plot energy scan results for N₂."""
    plt.figure(figsize=(12, 8))
    
    # Create main subplot for energies
    ax1 = plt.subplot(211)
    
    distances = results['distances']
    
    # Plot VQE energies
    ax1.plot(
        distances,
        results['vqe'],
        'o-',
        label='VQE (GPU-accelerated)',
        color='limegreen',
        linewidth=2,
        markersize=8
    )
    
    # Plot RHF energies
    if any(~np.isnan(results['rhf'])):
        ax1.plot(
            distances,
            results['rhf'],
            's--',
            label='RHF',
            color='red',
            linewidth=2,
            markersize=6
        )
    
    # Plot MP2 energies if available
    if any(~np.isnan(results['mp2'])):
        ax1.plot(
            distances,
            results['mp2'],
            'x-',
            label='MP2',
            color='purple',
            linewidth=2,
            markersize=6
        )
    
    # Plot CASSCF energies
    if any(~np.isnan(results['casscf'])):
        ax1.plot(
            distances,
            results['casscf'],
            'd-',
            label='CASSCF',
            color='blue',
            linewidth=2,
            markersize=6
        )
    
    # Plot styling for energy plot
    ax1.set_xlabel("N-N Distance (Å)", fontsize=14)
    ax1.set_ylabel("Energy (Hartree)", fontsize=14)
    ax1.set_title(f"N₂ Potential Energy Curve ({BASIS} basis)", fontsize=16)
    ax1.legend(fontsize=12)
    ax1.grid(True, alpha=0.3)
    
    # Add text box with equilibrium information
    try:
        vqe_min_idx = np.nanargmin(results['vqe'])
        eq_distance = distances[vqe_min_idx]
        min_energy = results['vqe'][vqe_min_idx]
        
        textstr = f"Equilibrium distance: {eq_distance:.3f} Å\n"
        textstr += f"VQE minimum energy: {min_energy:.6f} Ha"
        
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.4)
        ax1.text(0.05, 0.05, textstr, transform=ax1.transAxes, 
                 fontsize=12, verticalalignment='bottom', bbox=props)
    except:
        pass
    
    # Create second subplot for timing information
    ax2 = plt.subplot(212)
    
    # Plot VQE runtimes
    ax2.bar(distances, results['vqe_times'], width=0.05, alpha=0.7, color='blue')
    ax2.set_xlabel("N-N Distance (Å)", fontsize=14)
    ax2.set_ylabel("VQE Runtime (s)", fontsize=14)
    ax2.set_title("GPU-accelerated VQE Runtime per Distance", fontsize=16)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

def print_summary(results):
    """Get summary of N₂ results."""
    try:
        # Find minimum energies
        vqe_min_idx = np.nanargmin(results['vqe'])
        distances = results['distances']
        eq_distance = distances[vqe_min_idx]
        min_vqe = results['vqe'][vqe_min_idx]
        avg_time = np.nanmean(results['vqe_times'])
        
        summary = f"\n{'=' * 30} N₂ RESULTS SUMMARY {'=' * 30}\n"
        summary += f"Basis set: {BASIS}\n"
        summary += f"Active space: {sum(ACTIVE_ELECTRONS)}e in {ACTIVE_ORBITALS}o\n"
        summary += f"Average VQE runtime per point: {avg_time:.2f} seconds\n"
        summary += f"Equilibrium bond distance: {eq_distance:.3f} Å (exp: ~1.1 Å)\n"
        summary += f"VQE minimum energy: {min_vqe:.8f} Hartree\n"
        
        # Compare with reference values
        if not np.isnan(results['rhf'][vqe_min_idx]):
            rhf_energy = results['rhf'][vqe_min_idx]
            summary += f"RHF energy: {rhf_energy:.8f} Hartree\n"
            summary += f"Correlation energy: {min_vqe - rhf_energy:.8f} Hartree\n"
        
        if not np.isnan(results['mp2'][vqe_min_idx]):
            mp2_energy = results['mp2'][vqe_min_idx]
            summary += f"MP2 energy: {mp2_energy:.8f} Hartree\n"
            summary += f"VQE error vs MP2: {(min_vqe - mp2_energy) * 1000:.3f} mHartree\n"
            
        if not np.isnan(results['casscf'][vqe_min_idx]):
            casscf_energy = results['casscf'][vqe_min_idx]
            summary += f"CASSCF energy: {casscf_energy:.8f} Hartree\n"
            summary += f"VQE error vs CASSCF: {(min_vqe - casscf_energy) * 1000:.3f} mHartree\n"
        
        # For larger bond distances, compute dissociation energy
        try:
            large_dist_idx = -1  # Last point (largest separation)
            dissoc_energy = results['vqe'][large_dist_idx]
            de = (min_vqe - dissoc_energy) * 627.5  # Convert to kcal/mol
            summary += f"Dissociation energy: {-de:.2f} kcal/mol\n"
        except:
            pass
        
        summary += f"{'=' * 80}\n"
        return summary
    except Exception as e:
        return f"Error generating summary: {e}"


# Main execution
if __name__ == "__main__":
    print(f"N₂ simulation started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Run simulation
    results = run_n2_simulation()
    
    # Plot results
    plot_results(results, save_path="n2_vqe_gpu.png")
    
    # Print summary
    print(print_summary(results))
    
    print(f"Simulation completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
"o2"

In [ ]:
"""
GPU-Accelerated VQE for O₂ (Oxygen) Molecule
Optimized for qiskit-aer-gpu-cu11 0.17.0
"""
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime
import os

from qiskit.primitives import BackendEstimator
from qiskit_aer import AerSimulator
from qiskit_algorithms import VQE, NumPyMinimumEigensolver
from qiskit_algorithms.optimizers import L_BFGS_B

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

from pyscf import gto, scf, mcscf, mp

# Set number of OpenMP threads for PySCF
import os
os.environ["OMP_NUM_THREADS"] = "4"  # Adjust based on your CPU

# Configuration for O₂ molecule - a triplet!
BASIS = "6-31g"
ACTIVE_ELECTRONS = (6, 4)  # 10 valence electrons, triplet state (2 unpaired)
ACTIVE_ORBITALS = 8  # Include 2p orbitals
CORE_ORBITALS = [0, 1]  # Skip 1s core orbitals
MAX_ITER = 400
SPIN = 2  # Triplet state (spin multiplicity = 2S+1 = 3 -> S=1)

# Bond distances to scan (O-O bond in Angstrom)
DISTANCES = np.concatenate([
    np.linspace(1.0, 1.4, 7),  # Around equilibrium (1.21 Å)
    np.linspace(1.5, 2.5, 5)   # Dissociation region
])

def configure_gpu_backend():
    """Configure and test GPU backend."""
    try:
        backend = AerSimulator(
            method="statevector",
            device="GPU",
            precision="single",  # Use single precision for speed
            max_parallel_threads=0,  # Let CUDA decide
            max_parallel_experiments=1,  # For statevector, usually 1 is best
            cuStateVec_enable=True  # Enable NVIDIA cuStateVec library if available
        )
        
        # Test GPU backend
        result = backend.run_circuits(backend.initialize(2, 0))
        
        # Success - GPU is working
        gpu_info = backend.available_devices()[0]
        print(f"GPU acceleration active: {gpu_info}")
        return backend, True
        
    except Exception as e:
        print(f"GPU acceleration failed: {e}")
        print("Falling back to CPU backend...")
        backend = AerSimulator(
            method="statevector",
            max_parallel_threads=int(os.environ.get("OMP_NUM_THREADS", 4))
        )
        return backend, False

def get_o2_geometry(distance):
    """Generate O₂ molecule geometry at given bond distance."""
    return f"O 0 0 {-distance/2}; O 0 0 {distance/2}"

def build_qiskit_problem(distance):
    """Build Qiskit Nature problem with proper active space."""
    atom_str = get_o2_geometry(distance)
    
    # Create driver and run - note the spin=2 for triplet state
    driver = PySCFDriver(
        atom=atom_str, 
        basis=BASIS, 
        unit=DistanceUnit.ANGSTROM, 
        charge=0, 
        spin=SPIN  # Triplet state (2 unpaired electrons)
    )
    problem = driver.run()
    
    # Print full system information
    print(f"Full system: {problem.num_spatial_orbitals} orbitals, "
          f"{problem.num_particles} electrons (triplet state)")
    
    # Calculate active orbitals list
    max_orbital = CORE_ORBITALS[-1] + ACTIVE_ORBITALS + 1
    active_orbitals_list = list(range(CORE_ORBITALS[-1] + 1, max_orbital))
    
    # Apply active space transformer with explicit orbital selection
    transformer = ActiveSpaceTransformer(
        num_electrons=ACTIVE_ELECTRONS,
        num_spatial_orbitals=ACTIVE_ORBITALS,
        active_orbitals=active_orbitals_list
    )
    problem = transformer.transform(problem)
    
    print(f"Active space: {sum(problem.num_particles)} electrons in "
          f"{problem.num_spatial_orbitals} orbitals")
    
    return problem

def run_vqe(problem, backend, use_gpu=True):
    """Run VQE with optimal settings for O₂."""
    # Use parity mapper with symmetry reduction
    mapper = ParityMapper(num_particles=problem.num_particles)
    
    # Create initial state and ansatz
    init_state = HartreeFock(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper
    )
    
    ansatz = UCCSD(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
        initial_state=init_state
    )
    
    print(f"UCCSD parameters: {ansatz.num_parameters}")
    
    # Better initialization - small random values with seed for reproducibility
    np.random.seed(42)
    initial_point = 0.1 * np.random.random(ansatz.num_parameters)
    
    # L-BFGS-B works well for O₂'s complex electronic structure
    optimizer = L_BFGS_B(maxiter=MAX_ITER)
    
    # Map to qubit operator
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    
    # Create GPU-accelerated estimator if enabled
    if use_gpu:
        estimator = BackendEstimator(backend)
    else:
        from qiskit.primitives import Estimator
        estimator = Estimator()  # Default CPU estimator
    
    # Create and run VQE
    vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point)
    
    print(f"Qubit operator size: {qubit_op.num_qubits} qubits")
    print(f"Starting VQE optimization...")
    
    # Run VQE and time it
    start_time = time.time()
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    runtime = time.time() - start_time
    
    # Get interpreted energy
    energy = problem.interpret(result).total_energies[0].real
    
    print(f"VQE completed in {runtime:.1f} seconds")
    print(f"VQE energy: {energy:.8f} Hartree")
    print(f"Optimizer evaluations: {result.optimizer_evals}")
    
    return energy, runtime, result

def run_reference_calcs(distance):
    """Run reference classical calculations."""
    atom_str = get_o2_geometry(distance)
    
    # Create molecule - triplet state
    mol = gto.Mole()
    mol.atom = atom_str
    mol.basis = BASIS
    mol.charge = 0
    mol.spin = SPIN  # Triplet oxygen
    mol.build()
    
    results = {}
    
    # Run UHF for triplet state
    try:
        mf = scf.UHF(mol)  # Unrestricted HF for open-shell system
        results["uhf"] = mf.kernel()
        mf_obj = mf
    except Exception as e:
        print(f"UHF calculation failed: {e}")
        results["uhf"] = np.nan
        mf_obj = None
    
    # Run MP2 for dynamic correlation
    if mf_obj is not None:
        try:
            mp2 = mp.UMP2(mf_obj)  # Unrestricted MP2
            mp2.kernel()
            # MP2 correlation energy + UHF energy
            results["mp2"] = mp2.e_corr + results["uhf"]
        except Exception as e:
            print(f"MP2 calculation failed: {e}")
            results["mp2"] = np.nan
    
    # Run CASSCF with same active space
    try:
        mf_cas = scf.UHF(mol).run()  # Start with UHF for triplet
        mycas = mcscf.CASSCF(mf_cas, 
                           ncas=ACTIVE_ORBITALS, 
                           nelecas=sum(ACTIVE_ELECTRONS))
        results["casscf"] = mycas.kernel()[0]
    except Exception as e:
        print(f"CASSCF calculation failed: {e}")
        results["casscf"] = np.nan
    
    return results

def run_o2_simulation():
    """Run full O₂ simulation with VQE and classical references."""
    print(f"\n{'='*60}")
    print(f"GPU-Accelerated VQE for O₂ (Oxygen Molecule - Triplet State)")
    print(f"{'='*60}")
    print(f"Basis set: {BASIS}")
    print(f"Active space: {sum(ACTIVE_ELECTRONS)}e, {ACTIVE_ORBITALS}o")
    print(f"Spin: {SPIN} (Triplet state)")
    print(f"{'='*60}\n")
    
    # Configure GPU backend
    backend, is_gpu = configure_gpu_backend()
    
    vqe_energies = []
    vqe_times = []
    uhf_energies = []
    mp2_energies = []
    casscf_energies = []
    
    for i, dist in enumerate(DISTANCES):
        print(f"\n{'-'*30}")
        print(f"O-O Distance: {dist:.3f} Å ({i+1}/{len(DISTANCES)})")
        print(f"{'-'*30}")
        
        # Run reference calculations
        ref_results = run_reference_calcs(dist)
        uhf_energies.append(ref_results.get("uhf", np.nan))
        mp2_energies.append(ref_results.get("mp2", np.nan))
        casscf_energies.append(ref_results.get("casscf", np.nan))
        
        # Run VQE
        try:
            problem = build_qiskit_problem(dist)
            vqe_energy, vqe_time, _ = run_vqe(problem, backend, use_gpu=is_gpu)
            vqe_energies.append(vqe_energy)
            vqe_times.append(vqe_time)
            
            # Print comparison
            print("\nEnergy comparison at O-O distance = {:.3f} Å:".format(dist))
            print(f"  VQE:    {vqe_energy:.8f} Hartree (in {vqe_time:.1f}s)")
            print(f"  UHF:    {ref_results.get('uhf', 'N/A')}")
            if ref_results.get("mp2") is not None:
                print(f"  MP2:    {ref_results.get('mp2'):.8f} Hartree")
            if ref_results.get("casscf") is not None:
                print(f"  CASSCF: {ref_results.get('casscf'):.8f} Hartree")
            
            # Calculate correlation energy
            if not np.isnan(ref_results.get("uhf", np.nan)):
                corr_energy = vqe_energy - ref_results.get("uhf")
                print(f"  Correlation energy: {corr_energy:.8f} Hartree")
                
        except Exception as e:
            print(f"VQE calculation failed: {e}")
            vqe_energies.append(np.nan)
            vqe_times.append(np.nan)
    
    # Store results
    results = {
        'distances': DISTANCES,
        'vqe': vqe_energies,
        'vqe_times': vqe_times,
        'uhf': uhf_energies,  # Note: UHF for triplet
        'mp2': mp2_energies,
        'casscf': casscf_energies,
    }
    
    return results

def plot_results(results, save_path=None):
    """Plot energy scan results for O₂."""
    plt.figure(figsize=(12, 8))
    
    # Create main subplot for energies
    ax1 = plt.subplot(211)
    
    distances = results['distances']
    
    # Plot VQE energies
    ax1.plot(
        distances,
        results['vqe'],
        'o-',
        label='VQE (GPU-accelerated)',
        color='limegreen',
        linewidth=2,
        markersize=8
    )
    
    # Plot UHF energies
    if any(~np.isnan(results['uhf'])):
        ax1.plot(
            distances,
            results['uhf'],
            's--',
            label='UHF (triplet)',
            color='red',
            linewidth=2,
            markersize=6
        )
    
    # Plot MP2 energies if available
    if any(~np.isnan(results['mp2'])):
        ax1.plot(
            distances,
            results['mp2'],
            'x-',
            label='MP2',
            color='purple',
            linewidth=2,
            markersize=6
        )
    
    # Plot CASSCF energies
    if any(~np.isnan(results['casscf'])):
        ax1.plot(
            distances,
            results['casscf'],
            'd-',
            label='CASSCF',
            color='blue',
            linewidth=2,
            markersize=6
        )
    
    # Plot styling for energy plot
    ax1.set_xlabel("O-O Distance (Å)", fontsize=14)
    ax1.set_ylabel("Energy (Hartree)", fontsize=14)
    ax1.set_title(f"O₂ (Triplet) Potential Energy Curve ({BASIS} basis)", fontsize=16)
    ax1.legend(fontsize=12)
    ax1.grid(True, alpha=0.3)
    
    # Add text box with equilibrium information
    try:
        vqe_min_idx = np.nanargmin(results['vqe'])
        eq_distance = distances[vqe_min_idx]
        min_energy = results['vqe'][vqe_min_idx]
        
        textstr = f"Equilibrium distance: {eq_distance:.3f} Å\n"
        textstr += f"VQE minimum energy: {min_energy:.6f} Ha"
        
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.4)
        ax1.text(0.05, 0.05, textstr, transform=ax1.transAxes, 
                 fontsize=12, verticalalignment='bottom', bbox=props)
    except:
        pass
    
    # Create second subplot for timing information
    ax2 = plt.subplot(212)
    
    # Plot VQE runtimes
    ax2.bar(distances, results['vqe_times'], width=0.05, alpha=0.7, color='blue')
    ax2.set_xlabel("O-O Distance (Å)", fontsize=14)
    ax2.set_ylabel("VQE Runtime (s)", fontsize=14)
    ax2.set_title("GPU-accelerated VQE Runtime per Distance", fontsize=16)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

def print_summary(results):
    """Get summary of O₂ results."""
    try:
        # Find minimum energies
        vqe_min_idx = np.nanargmin(results['vqe'])
        distances = results['distances']
        eq_distance = distances[vqe_min_idx]
        min_vqe = results['vqe'][vqe_min_idx]
        avg_time = np.nanmean(results['vqe_times'])
        
        summary = f"\n{'=' * 30} O₂ RESULTS SUMMARY {'=' * 30}\n"
        summary += f"Basis set: {BASIS}\n"
        summary += f"Active space: {sum(ACTIVE_ELECTRONS)}e in {ACTIVE_ORBITALS}o\n"
        summary += f"Spin state: Triplet (spin = {SPIN})\n"
        summary += f"Average VQE runtime per point: {avg_time:.2f} seconds\n"
        summary += f"Equilibrium bond distance: {eq_distance:.3f} Å (exp: ~1.21 Å)\n"
        summary += f"VQE minimum energy: {min_vqe:.8f} Hartree\n"
        
        # Compare with reference values
        if not np.isnan(results['uhf'][vqe_min_idx]):
            uhf_energy = results['uhf'][vqe_min_idx]
            summary += f"UHF energy: {uhf_energy:.8f} Hartree\n"
            summary += f"Correlation energy: {min_vqe - uhf_energy:.8f} Hartree\n"
        
        if not np.isnan(results['mp2'][vqe_min_idx]):
            mp2_energy = results['mp2'][vqe_min_idx]
            summary += f"MP2 energy: {mp2_energy:.8f} Hartree\n"
            summary += f"VQE error vs MP2: {(min_vqe - mp2_energy) * 1000:.3f} mHartree\n"
            
        if not np.isnan(results['casscf'][vqe_min_idx]):
            casscf_energy = results['casscf'][vqe_min_idx]
            summary += f"CASSCF energy: {casscf_energy:.8f} Hartree\n"
            summary += f"VQE error vs CASSCF: {(min_vqe - casscf_energy) * 1000:.3f} mHartree\n"
        
        # For larger bond distances, compute dissociation energy
        try:
            large_dist_idx = -1  # Last point (largest separation)
            dissoc_energy = results['vqe'][large_dist_idx]
            de = (min_vqe - dissoc_energy) * 627.5  # Convert to kcal/mol
            summary += f"Dissociation energy: {-de:.2f} kcal/mol\n"
        except:
            pass
        
        summary += f"{'=' * 80}\n"
        return summary
    except Exception as e:
        return f"Error generating summary: {e}"


# Main execution
if __name__ == "__main__":
    print(f"O₂ simulation started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Run simulation
    results = run_o2_simulation()
    
    # Plot results
    plot_results(results, save_path="o2_vqe_gpu.png")
    
    # Print summary
    print(print_summary(results))
    
    print(f"Simulation completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
"c2h2"

In [ ]:
"""
GPU-Accelerated VQE for C₂H₂ (Acetylene) Molecule
Optimized for qiskit-aer-gpu-cu11 0.17.0
"""
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime
import os

from qiskit.primitives import BackendEstimator
from qiskit_aer import AerSimulator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP, L_BFGS_B

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

from pyscf import gto, scf, mcscf, mp

# Set number of OpenMP threads for PySCF
import os
os.environ["OMP_NUM_THREADS"] = "4"  # Adjust based on your CPU

# Configuration for C₂H₂ molecule
BASIS = "6-31g"  # Better basis for triple bond
ACTIVE_ELECTRONS = (5, 5)  # 10 valence electrons
ACTIVE_ORBITALS = 10  # Active orbitals for triple bond + C-H bonds
CORE_ORBITALS = [0, 1, 2, 3]  # Skip 1s core orbitals (2 for each C)
MAX_ITER = 400
CH_DISTANCE = 1.06  # C-H bond length

# Bond distances to scan (C-C bond in Angstrom)
DISTANCES = np.concatenate([
    np.linspace(1.0, 1.3, 7),  # Around equilibrium (1.2 Å)
    np.linspace(1.4, 2.0, 3)   # Dissociation region
])

def configure_gpu_backend():
    """Configure and test GPU backend."""
    try:
        backend = AerSimulator(
            method="statevector",
            device="GPU",
            precision="single",  # Use single precision for speed
            max_parallel_threads=0,  # Let CUDA decide
            max_parallel_experiments=1,  # For statevector, usually 1 is best
            cuStateVec_enable=True  # Enable NVIDIA cuStateVec library if available
        )
        
        # Test GPU backend
        result = backend.run_circuits(backend.initialize(2, 0))
        
        # Success - GPU is working
        gpu_info = backend.available_devices()[0]
        print(f"GPU acceleration active: {gpu_info}")
        return backend, True
        
    except Exception as e:
        print(f"GPU acceleration failed: {e}")
        print("Falling back to CPU backend...")
        backend = AerSimulator(
            method="statevector",
            max_parallel_threads=int(os.environ.get("OMP_NUM_THREADS", 4))
        )
        return backend, False

def get_c2h2_geometry(cc_distance, ch_distance=CH_DISTANCE):
    """Generate C₂H₂ molecule geometry at given C≡C bond distance."""
    return (f"C 0 0 {-cc_distance/2}; "
            f"C 0 0 {cc_distance/2}; "
            f"H 0 0 {-cc_distance/2 - ch_distance}; "
            f"H 0 0 {cc_distance/2 + ch_distance}")

def build_qiskit_problem(cc_distance):
    """Build Qiskit Nature problem with proper active space."""
    atom_str = get_c2h2_geometry(cc_distance)
    
    # Create driver and run
    driver = PySCFDriver(
        atom=atom_str, 
        basis=BASIS, 
        unit=DistanceUnit.ANGSTROM, 
        charge=0, 
        spin=0
    )
    problem = driver.run()
    
    # Print full system information
    print(f"Full system: {problem.num_spatial_orbitals} orbitals, "
          f"{problem.num_particles} electrons")
    
    # Calculate active orbitals list
    max_orbital = CORE_ORBITALS[-1] + ACTIVE_ORBITALS + 1
    active_orbitals_list = list(range(CORE_ORBITALS[-1] + 1, max_orbital))
    
    # Apply active space transformer with explicit orbital selection
    transformer = ActiveSpaceTransformer(
        num_electrons=ACTIVE_ELECTRONS,
        num_spatial_orbitals=ACTIVE_ORBITALS,
        active_orbitals=active_orbitals_list
    )
    problem = transformer.transform(problem)
    
    print(f"Active space: {sum(problem.num_particles)} electrons in "
          f"{problem.num_spatial_orbitals} orbitals")
    
    return problem

def run_vqe(problem, backend, use_gpu=True):
    """Run VQE with optimal settings for C₂H₂."""
    # Use parity mapper with symmetry reduction
    mapper = ParityMapper(num_particles=problem.num_particles)
    
    # Create initial state and ansatz
    init_state = HartreeFock(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper
    )
    
    ansatz = UCCSD(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
        initial_state=init_state
    )
    
    print(f"UCCSD parameters: {ansatz.num_parameters}")
    
    # Better initialization - small random values with seed for reproducibility
    np.random.seed(42)
    initial_point = 0.1 * np.random.random(ansatz.num_parameters)
    
    # SLSQP works well for acetylene
    optimizer = SLSQP(maxiter=MAX_ITER)
    
    # Map to qubit operator
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    
    # Create GPU-accelerated estimator if enabled
    if use_gpu:
        estimator = BackendEstimator(backend)
    else:
        from qiskit.primitives import Estimator
        estimator = Estimator()  # Default CPU estimator
    
    # Create and run VQE
    vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point)
    
    print(f"Qubit operator size: {qubit_op.num_qubits} qubits")
    print(f"Starting VQE optimization...")
    
    # Run VQE and time it
    start_time = time.time()
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    runtime = time.time() - start_time
    
    # Get interpreted energy
    energy = problem.interpret(result).total_energies[0].real
    
    print(f"VQE completed in {runtime:.1f} seconds")
    print(f"VQE energy: {energy:.8f} Hartree")
    print(f"Optimizer evaluations: {result.optimizer_evals}")
    
    return energy, runtime, result

def run_reference_calcs(cc_distance):
    """Run reference classical calculations."""
    atom_str = get_c2h2_geometry(cc_distance)
    
    # Create molecule
    mol = gto.Mole()
    mol.atom = atom_str
    mol.basis = BASIS
    mol.charge = 0
    mol.spin = 0
    mol.build()
    
    results = {}
    
    # Run RHF
    try:
        mf = scf.RHF(mol)
        results["rhf"] = mf.kernel()
        mf_obj = mf
    except Exception as e:
        print(f"RHF calculation failed: {e}")
        results["rhf"] = np.nan
        mf_obj = None
    
    # Run MP2 for dynamic correlation
    if mf_obj is not None:
        try:
            mp2 = mp.MP2(mf_obj)
            mp2.kernel()
            # MP2 correlation energy + RHF energy
            results["mp2"] = mp2.e_corr + results["rhf"]
        except Exception as e:
            print(f"MP2 calculation failed: {e}")
            results["mp2"] = np.nan
    
    # Run CASSCF with same active space
    try:
        mf_cas = scf.RHF(mol).run()
        mycas = mcscf.CASSCF(mf_cas, 
                            ncas=ACTIVE_ORBITALS, 
                            nelecas=sum(ACTIVE_ELECTRONS))
        results["casscf"] = mycas.kernel()[0]
    except Exception as e:
        print(f"CASSCF calculation failed: {e}")
        results["casscf"] = np.nan
    
    return results

def run_c2h2_simulation():
    """Run full C₂H₂ simulation with VQE and classical references."""
    print(f"\n{'='*60}")
    print(f"GPU-Accelerated VQE for C₂H₂ (Acetylene Molecule)")
    print(f"{'='*60}")
    print(f"Basis set: {BASIS}")
    print(f"Active space: {sum(ACTIVE_ELECTRONS)}e, {ACTIVE_ORBITALS}o")
    print(f"{'='*60}\n")
    
    # Configure GPU backend
    backend, is_gpu = configure_gpu_backend()
    
    vqe_energies = []
    vqe_times = []
    rhf_energies = []
    mp2_energies = []
    casscf_energies = []
    
    for i, dist in enumerate(DISTANCES):
        print(f"\n{'-'*30}")
        print(f"C≡C Distance: {dist:.3f} Å ({i+1}/{len(DISTANCES)})")
        print(f"{'-'*30}")
        
        # Run reference calculations
        ref_results = run_reference_calcs(dist)
        rhf_energies.append(ref_results.get("rhf", np.nan))
        mp2_energies.append(ref_results.get("mp2", np.nan))
        casscf_energies.append(ref_results.get("casscf", np.nan))
        
        # Run VQE
        try:
            problem = build_qiskit_problem(dist)
            vqe_energy, vqe_time, _ = run_vqe(problem, backend, use_gpu=is_gpu)
            vqe_energies.append(vqe_energy)
            vqe_times.append(vqe_time)
            
            # Print comparison
            print("\nEnergy comparison at C≡C distance = {:.3f} Å:".format(dist))
            print(f"  VQE:    {vqe_energy:.8f} Hartree (in {vqe_time:.1f}s)")
            print(f"  RHF:    {ref_results.get('rhf', 'N/A')}")
            if ref_results.get("mp2") is not None:
                print(f"  MP2:    {ref_results.get('mp2'):.8f} Hartree")
            if ref_results.get("casscf") is not None:
                print(f"  CASSCF: {ref_results.get('casscf'):.8f} Hartree")
            
            # Calculate correlation energy
            if not np.isnan(ref_results.get("rhf", np.nan)):
                corr_energy = vqe_energy - ref_results.get("rhf")
                print(f"  Correlation energy: {corr_energy:.8f} Hartree")
                
        except Exception as e:
            print(f"VQE calculation failed: {e}")
            vqe_energies.append(np.nan)
            vqe_times.append(np.nan)
    
    # Store results
    results = {
        'distances': DISTANCES,
        'vqe': vqe_energies,
        'vqe_times': vqe_times,
        'rhf': rhf_energies,
        'mp2': mp2_energies,
        'casscf': casscf_energies,
    }
    
    return results

def plot_results(results, save_path=None):
    """Plot energy scan results for C₂H₂."""
    plt.figure(figsize=(12, 8))
    
    # Create main subplot for energies
    ax1 = plt.subplot(211)
    
    distances = results['distances']
    
    # Plot VQE energies
    ax1.plot(
        distances,
        results['vqe'],
        'o-',
        label='VQE (GPU-accelerated)',
        color='limegreen',
        linewidth=2,
        markersize=8
    )
    
    # Plot RHF energies
    if any(~np.isnan(results['rhf'])):
        ax1.plot(
            distances,
            results['rhf'],
            's--',
            label='RHF',
            color='red',
            linewidth=2,
            markersize=6
        )
    
    # Plot MP2 energies if available
    if any(~np.isnan(results['mp2'])):
        ax1.plot(
            distances,
            results['mp2'],
            'x-',
            label='MP2',
            color='purple',
            linewidth=2,
            markersize=6
        )
    
    # Plot CASSCF energies
    if any(~np.isnan(results['casscf'])):
        ax1.plot(
            distances,
            results['casscf'],
            'd-',
            label='CASSCF',
            color='blue',
            linewidth=2,
            markersize=6
        )
    
    # Plot styling for energy plot
    ax1.set_xlabel("C≡C Distance (Å)", fontsize=14)
    ax1.set_ylabel("Energy (Hartree)", fontsize=14)
    ax1.set_title(f"C₂H₂ (Acetylene) Potential Energy Curve ({BASIS} basis)", fontsize=16)
    ax1.legend(fontsize=12)
    ax1.grid(True, alpha=0.3)
    
    # Add text box with equilibrium information
    try:
        vqe_min_idx = np.nanargmin(results['vqe'])
        eq_distance = distances[vqe_min_idx]
        min_energy = results['vqe'][vqe_min_idx]
        
        textstr = f"Equilibrium C≡C distance: {eq_distance:.3f} Å\n"
        textstr += f"VQE minimum energy: {min_energy:.6f} Ha"
        
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.4)
        ax1.text(0.05, 0.05, textstr, transform=ax1.transAxes, 
                 fontsize=12, verticalalignment='bottom', bbox=props)
    except:
        pass
    
    # Create second subplot for timing information
    ax2 = plt.subplot(212)
    
    # Plot VQE runtimes
    ax2.bar(distances, results['vqe_times'], width=0.05, alpha=0.7, color='blue')
    ax2.set_xlabel("C≡C Distance (Å)", fontsize=14)
    ax2.set_ylabel("VQE Runtime (s)", fontsize=14)
    ax2.set_title("GPU-accelerated VQE Runtime per Distance", fontsize=16)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

def print_summary(results):
    """Get summary of C₂H₂ results."""
    try:
        # Find minimum energies
        vqe_min_idx = np.nanargmin(results['vqe'])
        distances = results['distances']
        eq_distance = distances[vqe_min_idx]
        min_vqe = results['vqe'][vqe_min_idx]
        avg_time = np.nanmean(results['vqe_times'])
        
        summary = f"\n{'=' * 30} C₂H₂ RESULTS SUMMARY {'=' * 30}\n"
        summary += f"Basis set: {BASIS}\n"
        summary += f"Active space: {sum(ACTIVE_ELECTRONS)}e in {ACTIVE_ORBITALS}o\n"
        summary += f"Average VQE runtime per point: {avg_time:.2f} seconds\n"
        summary += f"Equilibrium C≡C bond distance: {eq_distance:.3f} Å (exp: ~1.20 Å)\n"
        summary += f"VQE minimum energy: {min_vqe:.8f} Hartree\n"
        
        # Compare with reference values
        if not np.isnan(results['rhf'][vqe_min_idx]):
            rhf_energy = results['rhf'][vqe_min_idx]
            summary += f"RHF energy: {rhf_energy:.8f} Hartree\n"
            summary += f"Correlation energy: {min_vqe - rhf_energy:.8f} Hartree\n"
        
        if not np.isnan(results['mp2'][vqe_min_idx]):
            mp2_energy = results['mp2'][vqe_min_idx]
            summary += f"MP2 energy: {mp2_energy:.8f} Hartree\n"
            summary += f"VQE error vs MP2: {(min_vqe - mp2_energy) * 1000:.3f} mHartree\n"
            
        if not np.isnan(results['casscf'][vqe_min_idx]):
            casscf_energy = results['casscf'][vqe_min_idx]
            summary += f"CASSCF energy: {casscf_energy:.8f} Hartree\n"
            summary += f"VQE error vs CASSCF: {(min_vqe - casscf_energy) * 1000:.3f} mHartree\n"
        
        # For larger bond distances, compute dissociation energy
        try:
            large_dist_idx = -1  # Last point (largest separation)
            dissoc_energy = results['vqe'][large_dist_idx]
            de = (min_vqe - dissoc_energy) * 627.5  # Convert to kcal/mol
            summary += f"Dissociation energy: {-de:.2f} kcal/mol\n"
        except:
            pass
        
        summary += f"{'=' * 80}\n"
        return summary
    except Exception as e:
        return f"Error generating summary: {e}"


# Main execution
if __name__ == "__main__":
    print(f"C₂H₂ simulation started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Run simulation
    results = run_c2h2_simulation()
    
    # Plot results
    plot_results(results, save_path="c2h2_vqe_gpu.png")
    
    # Print summary
    print(print_summary(results))
    
    print(f"Simulation completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
"c2h4"

In [ ]:
"""
GPU-Accelerated VQE for C₂H₄ (Ethylene) Molecule
Optimized for qiskit-aer-gpu-cu11 0.17.0
"""
import numpy as np
import matplotlib.pyplot as plt
import time
from datetime import datetime
import os

from qiskit.primitives import BackendEstimator
from qiskit_aer import AerSimulator
from qiskit_algorithms import VQE
from qiskit_algorithms.optimizers import SLSQP

from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature.second_q.mappers import ParityMapper
from qiskit_nature.second_q.circuit.library import HartreeFock, UCCSD

from pyscf import gto, scf, mcscf, mp

# Set number of OpenMP threads for PySCF
import os
os.environ["OMP_NUM_THREADS"] = "4"  # Adjust based on your CPU

# Configuration for C₂H₄ molecule
BASIS = "6-31g"  # Better basis for double bond
ACTIVE_ELECTRONS = (6, 6)  # 12 valence electrons
ACTIVE_ORBITALS = 12  # Active orbitals for double bond + C-H bonds
CORE_ORBITALS = [0, 1, 2, 3]  # Skip 1s core orbitals (2 for each C)
MAX_ITER = 400
CH_DISTANCE = 1.09  # C-H bond length
H_ANGLE = 120.0  # H-C-H angle in degrees

# Bond distances to scan (C=C bond in Angstrom)
DISTANCES = np.concatenate([
    np.linspace(1.2, 1.45, 7),  # Around equilibrium (1.33 Å)
    np.linspace(1.5, 1.8, 3)    # Stretched bond region
])

def configure_gpu_backend():
    """Configure and test GPU backend."""
    try:
        backend = AerSimulator(
            method="statevector",
            device="GPU",
            precision="single",  # Use single precision for speed
            max_parallel_threads=0,  # Let CUDA decide
            max_parallel_experiments=1,  # For statevector, usually 1 is best
            cuStateVec_enable=True  # Enable NVIDIA cuStateVec library if available
        )
        
        # Test GPU backend
        result = backend.run_circuits(backend.initialize(2, 0))
        
        # Success - GPU is working
        gpu_info = backend.available_devices()[0]
        print(f"GPU acceleration active: {gpu_info}")
        return backend, True
        
    except Exception as e:
        print(f"GPU acceleration failed: {e}")
        print("Falling back to CPU backend...")
        backend = AerSimulator(
            method="statevector",
            max_parallel_threads=int(os.environ.get("OMP_NUM_THREADS", 4))
        )
        return backend, False

def get_c2h4_geometry(cc_distance, ch_distance=CH_DISTANCE, h_angle=H_ANGLE):
    """Generate planar ethylene geometry with given C=C bond distance."""
    # Convert angle to radians
    h_angle_rad = h_angle * np.pi / 180.0
    
    # Calculate H positions
    x_h = ch_distance * np.sin(h_angle_rad)
    y_h = 0.0  # Keep planar in xz plane
    z_h_offset = ch_distance * np.cos(h_angle_rad)
    
    return (f"C 0.0 0.0 {-cc_distance/2}; "
            f"C 0.0 0.0 {cc_distance/2}; "
            f"H {x_h} {y_h} {-cc_distance/2 - z_h_offset}; "
            f"H {-x_h} {y_h} {-cc_distance/2 - z_h_offset}; "
            f"H {x_h} {y_h} {cc_distance/2 + z_h_offset}; "
            f"H {-x_h} {y_h} {cc_distance/2 + z_h_offset}")

def build_qiskit_problem(cc_distance):
    """Build Qiskit Nature problem with proper active space."""
    atom_str = get_c2h4_geometry(cc_distance)
    
    # Create driver and run
    driver = PySCFDriver(
        atom=atom_str, 
        basis=BASIS, 
        unit=DistanceUnit.ANGSTROM, 
        charge=0, 
        spin=0
    )
    problem = driver.run()
    
    # Print full system information
    print(f"Full system: {problem.num_spatial_orbitals} orbitals, "
          f"{problem.num_particles} electrons")
    
    # Calculate active orbitals list
    max_orbital = CORE_ORBITALS[-1] + ACTIVE_ORBITALS + 1
    active_orbitals_list = list(range(CORE_ORBITALS[-1] + 1, max_orbital))
    
    # Apply active space transformer with explicit orbital selection
    transformer = ActiveSpaceTransformer(
        num_electrons=ACTIVE_ELECTRONS,
        num_spatial_orbitals=ACTIVE_ORBITALS,
        active_orbitals=active_orbitals_list
    )
    problem = transformer.transform(problem)
    
    print(f"Active space: {sum(problem.num_particles)} electrons in "
          f"{problem.num_spatial_orbitals} orbitals")
    
    return problem

def run_vqe(problem, backend, use_gpu=True):
    """Run VQE with optimal settings for C₂H₄."""
    # Use parity mapper with symmetry reduction
    mapper = ParityMapper(num_particles=problem.num_particles)
    
    # Create initial state and ansatz
    init_state = HartreeFock(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper
    )
    
    ansatz = UCCSD(
        problem.num_spatial_orbitals,
        problem.num_particles,
        mapper,
        initial_state=init_state
    )
    
    print(f"UCCSD parameters: {ansatz.num_parameters}")
    
    # Better initialization - small random values with seed for reproducibility
    np.random.seed(42)
    initial_point = 0.1 * np.random.random(ansatz.num_parameters)
    
    # SLSQP works well for ethylene
    optimizer = SLSQP(maxiter=MAX_ITER)
    
    # Map to qubit operator
    qubit_op = mapper.map(problem.hamiltonian.second_q_op())
    
    # Create GPU-accelerated estimator if enabled
    if use_gpu:
        estimator = BackendEstimator(backend)
    else:
        from qiskit.primitives import Estimator
        estimator = Estimator()  # Default CPU estimator
    
    # Create and run VQE
    vqe = VQE(estimator, ansatz, optimizer, initial_point=initial_point)
    
    print(f"Qubit operator size: {qubit_op.num_qubits} qubits")
    print(f"Starting VQE optimization...")
    
    # Run VQE and time it
    start_time = time.time()
    result = vqe.compute_minimum_eigenvalue(qubit_op)
    runtime = time.time() - start_time
    
    # Get interpreted energy
    energy = problem.interpret(result).total_energies[0].real
    
    print(f"VQE completed in {runtime:.1f} seconds")
    print(f"VQE energy: {energy:.8f} Hartree")
    print(f"Optimizer evaluations: {result.optimizer_evals}")
    
    return energy, runtime, result

def run_reference_calcs(cc_distance):
    """Run reference classical calculations."""
    atom_str = get_c2h4_geometry(cc_distance)
    
    # Create molecule
    mol = gto.Mole()
    mol.atom = atom_str
    mol.basis = BASIS
    mol.charge = 0
    mol.spin = 0
    mol.build()
    
    results = {}
    
    # Run RHF
    try:
        mf = scf.RHF(mol)
        results["rhf"] = mf.kernel()
        mf_obj = mf
    except Exception as e:
        print(f"RHF calculation failed: {e}")
        results["rhf"] = np.nan
        mf_obj = None
    
    # Run MP2 for dynamic correlation
    if mf_obj is not None:
        try:
            mp2 = mp.MP2(mf_obj)
            mp2.kernel()
            # MP2 correlation energy + RHF energy
            results["mp2"] = mp2.e_corr + results["rhf"]
        except Exception as e:
            print(f"MP2 calculation failed: {e}")
            results["mp2"] = np.nan
    
    # Run CASSCF with same active space
    try:
        mf_cas = scf.RHF(mol).run()
        mycas = mcscf.CASSCF(mf_cas, 
                            ncas=ACTIVE_ORBITALS, 
                            nelecas=sum(ACTIVE_ELECTRONS))
        results["casscf"] = mycas.kernel()[0]
    except Exception as e:
        print(f"CASSCF calculation failed: {e}")
        results["casscf"] = np.nan
    
    return results

def run_c2h4_simulation():
    """Run full C₂H₄ simulation with VQE and classical references."""
    print(f"\n{'='*60}")
    print(f"GPU-Accelerated VQE for C₂H₄ (Ethylene Molecule)")
    print(f"{'='*60}")
    print(f"Basis set: {BASIS}")
    print(f"Active space: {sum(ACTIVE_ELECTRONS)}e, {ACTIVE_ORBITALS}o")
    print(f"{'='*60}\n")
    
    # Configure GPU backend
    backend, is_gpu = configure_gpu_backend()
    
    vqe_energies = []
    vqe_times = []
    rhf_energies = []
    mp2_energies = []
    casscf_energies = []
    
    for i, dist in enumerate(DISTANCES):
        print(f"\n{'-'*30}")
        print(f"C=C Distance: {dist:.3f} Å ({i+1}/{len(DISTANCES)})")
        print(f"{'-'*30}")
        
        # Run reference calculations
        ref_results = run_reference_calcs(dist)
        rhf_energies.append(ref_results.get("rhf", np.nan))
        mp2_energies.append(ref_results.get("mp2", np.nan))
        casscf_energies.append(ref_results.get("casscf", np.nan))
        
        # Run VQE
        try:
            problem = build_qiskit_problem(dist)
            vqe_energy, vqe_time, _ = run_vqe(problem, backend, use_gpu=is_gpu)
            vqe_energies.append(vqe_energy)
            vqe_times.append(vqe_time)
            
            # Print comparison
            print("\nEnergy comparison at C=C distance = {:.3f} Å:".format(dist))
            print(f"  VQE:    {vqe_energy:.8f} Hartree (in {vqe_time:.1f}s)")
            print(f"  RHF:    {ref_results.get('rhf', 'N/A')}")
            if ref_results.get("mp2") is not None:
                print(f"  MP2:    {ref_results.get('mp2'):.8f} Hartree")
            if ref_results.get("casscf") is not None:
                print(f"  CASSCF: {ref_results.get('casscf'):.8f} Hartree")
            
            # Calculate correlation energy
            if not np.isnan(ref_results.get("rhf", np.nan)):
                corr_energy = vqe_energy - ref_results.get("rhf")
                print(f"  Correlation energy: {corr_energy:.8f} Hartree")
                
        except Exception as e:
            print(f"VQE calculation failed: {e}")
            vqe_energies.append(np.nan)
            vqe_times.append(np.nan)
    
    # Store results
    results = {
        'distances': DISTANCES,
        'vqe': vqe_energies,
        'vqe_times': vqe_times,
        'rhf': rhf_energies,
        'mp2': mp2_energies,
        'casscf': casscf_energies,
    }
    
    return results

def plot_results(results, save_path=None):
    """Plot energy scan results for C₂H₄."""
    plt.figure(figsize=(12, 8))
    
    # Create main subplot for energies
    ax1 = plt.subplot(211)
    
    distances = results['distances']
    
    # Plot VQE energies
    ax1.plot(
        distances,
        results['vqe'],
        'o-',
        label='VQE (GPU-accelerated)',
        color='limegreen',
        linewidth=2,
        markersize=8
    )
    
    # Plot RHF energies
    if any(~np.isnan(results['rhf'])):
        ax1.plot(
            distances,
            results['rhf'],
            's--',
            label='RHF',
            color='red',
            linewidth=2,
            markersize=6
        )
    
    # Plot MP2 energies if available
    if any(~np.isnan(results['mp2'])):
        ax1.plot(
            distances,
            results['mp2'],
            'x-',
            label='MP2',
            color='purple',
            linewidth=2,
            markersize=6
        )
    
    # Plot CASSCF energies
    if any(~np.isnan(results['casscf'])):
        ax1.plot(
            distances,
            results['casscf'],
            'd-',
            label='CASSCF',
            color='blue',
            linewidth=2,
            markersize=6
        )
    
    # Plot styling for energy plot
    ax1.set_xlabel("C=C Distance (Å)", fontsize=14)
    ax1.set_ylabel("Energy (Hartree)", fontsize=14)
    ax1.set_title(f"C₂H₄ (Ethylene) Potential Energy Curve ({BASIS} basis)", fontsize=16)
    ax1.legend(fontsize=12)
    ax1.grid(True, alpha=0.3)
    
    # Add text box with equilibrium information
    try:
        vqe_min_idx = np.nanargmin(results['vqe'])
        eq_distance = distances[vqe_min_idx]
        min_energy = results['vqe'][vqe_min_idx]
        
        textstr = f"Equilibrium C=C distance: {eq_distance:.3f} Å\n"
        textstr += f"VQE minimum energy: {min_energy:.6f} Ha"
        
        props = dict(boxstyle='round', facecolor='wheat', alpha=0.4)
        ax1.text(0.05, 0.05, textstr, transform=ax1.transAxes, 
                 fontsize=12, verticalalignment='bottom', bbox=props)
    except:
        pass
    
    # Create second subplot for timing information
    ax2 = plt.subplot(212)
    
    # Plot VQE runtimes
    ax2.bar(distances, results['vqe_times'], width=0.05, alpha=0.7, color='blue')
    ax2.set_xlabel("C=C Distance (Å)", fontsize=14)
    ax2.set_ylabel("VQE Runtime (s)", fontsize=14)
    ax2.set_title("GPU-accelerated VQE Runtime per Distance", fontsize=16)
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    if save_path:
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Plot saved to {save_path}")
    else:
        plt.show()

def print_summary(results):
    """Get summary of C₂H₄ results."""
    try:
        # Find minimum energies
        vqe_min_idx = np.nanargmin(results['vqe'])
        distances = results['distances']
        eq_distance = distances[vqe_min_idx]
        min_vqe = results['vqe'][vqe_min_idx]
        avg_time = np.nanmean(results['vqe_times'])
        
        summary = f"\n{'=' * 30} C₂H₄ RESULTS SUMMARY {'=' * 30}\n"
        summary += f"Basis set: {BASIS}\n"
        summary += f"Active space: {sum(ACTIVE_ELECTRONS)}e in {ACTIVE_ORBITALS}o\n"
        summary += f"Average VQE runtime per point: {avg_time:.2f} seconds\n"
        summary += f"Equilibrium C=C bond distance: {eq_distance:.3f} Å (exp: ~1.33 Å)\n"
        summary += f"VQE minimum energy: {min_vqe:.8f} Hartree\n"
        
        # Compare with reference values
        if not np.isnan(results['rhf'][vqe_min_idx]):
            rhf_energy = results['rhf'][vqe_min_idx]
            summary += f"RHF energy: {rhf_energy:.8f} Hartree\n"
            summary += f"Correlation energy: {min_vqe - rhf_energy:.8f} Hartree\n"
        
        if not np.isnan(results['mp2'][vqe_min_idx]):
            mp2_energy = results['mp2'][vqe_min_idx]
            summary += f"MP2 energy: {mp2_energy:.8f} Hartree\n"
            summary += f"VQE error vs MP2: {(min_vqe - mp2_energy) * 1000:.3f} mHartree\n"
            
        if not np.isnan(results['casscf'][vqe_min_idx]):
            casscf_energy = results['casscf'][vqe_min_idx]
            summary += f"CASSCF energy: {casscf_energy:.8f} Hartree\n"
            summary += f"VQE error vs CASSCF: {(min_vqe - casscf_energy) * 1000:.3f} mHartree\n"
        
        # For larger bond distances, compute dissociation energy
        try:
            large_dist_idx = -1  # Last point (largest separation)
            dissoc_energy = results['vqe'][large_dist_idx]
            de = (min_vqe - dissoc_energy) * 627.5  # Convert to kcal/mol
            summary += f"Dissociation energy: {-de:.2f} kcal/mol\n"
        except:
            pass
        
        summary += f"{'=' * 80}\n"
        return summary
    except Exception as e:
        return f"Error generating summary: {e}"


# Main execution
if __name__ == "__main__":
    print(f"C₂H₄ simulation started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Run simulation
    results = run_c2h4_simulation()
    
    # Plot results
    plot_results(results, save_path="c2h4_vqe_gpu.png")
    
    # Print summary
    print(print_summary(results))
    
    print(f"Simulation completed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")